In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder

In [2]:
df = pd.read_csv("Traffic_Crashes_Resulting_in_Injury_20240518.csv")
display(df.iloc[:, 3])
display(df.iloc[:, 15])

C:\Users\emily\AppData\Local\Temp\ipykernel_14540\3174712082.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Traffic_Crashes_Resulting_in_Injury_20240518.csv")


0        180085641
1        220672946
2        130379165
3        220527222
4          4395868
           ...    
59004    230392695
59005    230713386
59006    230347242
59007    230886886
59008    230728169
Name: case_id_pkey, Length: 59009, dtype: object

0        3801
1        3801
2        3801
3        3801
4        3801
         ... 
59004    3801
59005    3801
59006    3801
59007    3801
59008    3801
Name: juris, Length: 59009, dtype: object

In [3]:
df = df.drop(columns=['cnn_intrsctn_fkey', 'cnn_sgmt_fkey', 'case_id_pkey', 
                      'tb_latitude', 'tb_longitude', 'geocode_source', 
                      'officer_id', 'beat_number', 'direction', 
                      'control_device', 'vz_pcf_group', 'vz_pcf_description', 
                      'vz_pcf_link', 'number_killed', 'number_injured', 
                      'street_view', 'dph_col_grp', 'party_at_fault',
                      'party1_dir_of_travel', 'party1_move_pre_acc', 
                      'party2_dir_of_travel', 'party2_move_pre_acc', 
                      'point', 'data_as_of', 'data_updated_at', 
                      'data_loaded_at', 'Neighborhoods', 'SF Find Neighborhoods', 
                      'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods',
                      'reporting_district', "primary_rd", "secondary_rd", "unique_id", "juris"])
df = df.dropna(axis="index", subset=["collision_time"])
df = df.dropna(axis="index", subset=["intersection"])
df = df.dropna(axis="index", subset=["distance"])

In [4]:

# display(df["road_cond_1"].unique())
# display(df["road_cond_2"].unique())
df["road_cond_1"].where(df["road_cond_1"] != "Holes, Deep Rut", other="Holes, Deep Ruts", inplace=True)
df["road_cond_1"].where(df["road_cond_1"] != "No Unusual Condition", other="None", inplace=True)
df["road_cond_1"].where(df["road_cond_1"] != "Not Stated", other="None", inplace=True)
df["road_cond_2"].where(df["road_cond_2"] != "No Unusual Condition", other="None", inplace=True)
df["road_cond_2"].where(df["road_cond_2"] != "Not Stated", other="None", inplace=True)
df["vz_pcf_code"].where(~df["vz_pcf_code"].isnull(), other="Unknown", inplace=True)
# df["party2_type"].where(df["party2_type"])
# display(df["road_cond_1"].unique())
# display(df["road_cond_2"].unique())
df["road_cond_comb"] = df["road_cond_1"] + ";" + df["road_cond_2"]
df["road_cond_comb"] = df["road_cond_comb"].str.split(";")
df = df.drop(columns=['road_cond_1', 'road_cond_2'])
# df.loc[:, "juris"] = df.loc[:, "juris"].astype(str)
df["collision_datetime"] = pd.to_datetime(df["collision_datetime"], format= "%m/%d/%Y %H:%M:%S %p")
df.set_index("collision_datetime", inplace=True)
display(df)

,geocode_location,collision_date,collision_time,accident_year,month,day_of_week,time_cat,distance,weather_1,weather_2,...,mviw,ped_action,road_surface,lighting,intersection,vz_pcf_code,dph_col_grp_description,party1_type,party2_type,road_cond_comb
collision_datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-26 05:45:00,CITY STREET,2018 January 26,17:45:00,2018,January,Friday,2:01 pm to 6:00 pm,0.0,Other,Not Stated,...,Other Motor Vehicle,Not Stated,Dry,Dark - Street Lights,Intersection <= 20ft,Unknown,Vehicle(s) Only Involved,Driver,Driver,"[None, None]"
2022-10-01 02:27:00,CITY STREET,2022 October 01,14:27:00,2022,October,Saturday,2:01 pm to 6:00 pm,121.0,Clear,Not Stated,...,Bicycle,No Pedestrian Involved,Dry,Daylight,Midblock > 20ft,22350,Vehicle-Bicycle,Bicyclist,Driver,"[None, None]"
2013-05-08 10:51:00,CITY STREET,2013 May 08,22:51:00,2013,May,Wednesday,10:01 pm to 2:00 am,0.0,Clear,Not Stated,...,Pedestrian,Crossing in Crosswalk at Intersection,Dry,Dark - Street Lights,Intersection <= 20ft,Unknown,Vehicle-Pedestrian,Driver,Pedestrian,"[None, None]"
2022-08-07 10:18:00,CITY STREET,2022 August 07,22:18:00,2022,August,Sunday,10:01 pm to 2:00 am,228.0,Clear,Not Stated,...,Fixed Object,No Pedestrian Involved,Dry,Dark - Street Lights,Midblock > 20ft,Unknown,Bicycle Only,Bicyclist,NaN,"[Obstruction on Roadway, Other]"
2009-09-03 01:35:00,CITY STREET,2009 September 03,13:35:00,2009,September,Thursday,10:01 am to 2:00 pm,33.0,Clear,Not Stated,...,Parked Motor Vehicle,No Pedestrian Involved,Dry,Daylight,Intersection Rear End <= 150ft,Unknown,Vehicle(s) Only Involved,Driver,Parked Vehicle,"[None, None]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 02:10:00,CITY STREET,2023 June 07,14:10:00,2023,June,Wednesday,2:01 pm to 6:00 pm,0.0,Clear,Not Stated,...,Fixed Object,No Pedestrian Involved,Dry,Daylight,Intersection <= 20ft,22350,Vehicle(s) Only Involved,Driver,NaN,"[None, None]"
2023-10-04 02:00:00,CITY STREET,2023 October 04,14:00:00,2023,October,Wednesday,10:01 am to 2:00 pm,0.0,Clear,Not Stated,...,Other Motor Vehicle,No Pedestrian Involved,Dry,Daylight,Intersection <= 20ft,21453(a),Vehicle(s) Only Involved,Driver,Driver,"[None, None]"
2023-05-19 09:40:00,CITY STREET,2023 May 19,21:40:00,2023,May,Friday,6:01 pm to 10:00 pm,0.0,Not Stated,Not Stated,...,Other Motor Vehicle,No Pedestrian Involved,Dry,Dark - Street Lights,Intersection <= 20ft,21801(a),Vehicle(s) Only Involved,Driver,Driver,"[None, None]"


In [5]:
df.columns

Index(['geocode_location', 'collision_date', 'collision_time', 'accident_year',
       'month', 'day_of_week', 'time_cat', 'distance', 'weather_1',
       'weather_2', 'collision_severity', 'type_of_collision', 'mviw',
       'ped_action', 'road_surface', 'lighting', 'intersection', 'vz_pcf_code',
       'dph_col_grp_description', 'party1_type', 'party2_type',
       'road_cond_comb'],
      dtype='object')

In [6]:
df["road_cond_comb"]

collision_datetime
2018-01-26 05:45:00                           [None, None]
2022-10-01 02:27:00                           [None, None]
2013-05-08 10:51:00                           [None, None]
2022-08-07 10:18:00        [Obstruction on Roadway, Other]
2009-09-03 01:35:00                           [None, None]
                                      ...                 
2023-06-07 02:10:00                           [None, None]
2023-10-04 02:00:00                           [None, None]
2023-05-19 09:40:00                           [None, None]
2023-12-19 08:11:00                           [None, None]
2023-10-10 03:25:00    [Construction or Repair Zone, None]
Name: road_cond_comb, Length: 58872, dtype: object

In [7]:
mlb = MultiLabelBinarizer()
mlb.fit_transform(df["road_cond_comb"])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [8]:
df.isnull().sum()

geocode_location              0
collision_date                0
collision_time                0
accident_year                 0
month                         0
day_of_week                   8
time_cat                      0
distance                      0
weather_1                     0
weather_2                   793
collision_severity            0
type_of_collision             0
mviw                          0
ped_action                    0
road_surface                  0
lighting                      0
intersection                  0
vz_pcf_code                   0
dph_col_grp_description       0
party1_type                   8
party2_type                4204
road_cond_comb                0
dtype: int64

In [9]:
df["party2_type"].unique()

array(['Driver', 'Pedestrian', nan, 'Parked Vehicle', 'Bicyclist',
       'Other', 'Not Stated'], dtype=object)

In [10]:
df["day_of_week"].unique()

array(['Friday', 'Saturday', 'Wednesday', 'Sunday', 'Thursday', 'Tuesday',
       'Monday', nan], dtype=object)

In [11]:
df["vz_pcf_code"].unique()

array(['Unknown', '22350', '22106', 'Other than driver', '22107',
       'Other Than Driver', 'Other Improper Driving',
       'Other improper driving', '21801(a)', '21755(a)', '21658(a)',
       'Not Stated', '23152(a)', '22517', '21963', '22109', '21453(a)',
       '22450(a)', '21750', '21650', '21751', '21703', '22411',
       '21453(d)', '21950(a)', '21456(b)', '22101(d)', '21663', '22103',
       '7.2.12TC', '21955', '21657', '21802(a)', '21650.1', '22500',
       '21453(c)', '22100(a)', '21456(c)', '21950(b)', '23123(a)',
       '21802(b)', '22515', '21804(a)', '24400(b)', '21651(a)(1)',
       '21704(a)', '23153(a)', '22102', '21802(a,b)', '21954(a)',
       '21801(b)', '24002(a)', '21800(a)', '21460(a)', '21651(b)',
       '21235(g)', '20001(a)', '22100(b)', '21229(b)', '22351(a)',
       '21461.5', '21717', '22106(a)', '21202(a)', '21806(a)', '21800(b)',
       '21209(a)', '23153(b)', '21956(a)', '21651(a)', '21800(d)',
       '21453(b)', '22515(a)', '21456(a)', '21952', '2175